<a href="https://colab.research.google.com/github/Rajeswari0410/Movie-recommendation-system/blob/main/project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Movie Recommendation system


In [ ]:
import numpy as np
import pandas as pd
import warnings

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
columns_name = ['user_id','item_id','rating','timestamp']
df = pd.read_csv('/content/gdrive/MyDrive/RWDS/ml-100k/ml-100k/u.data',sep = '\t',names = columns_name)

In [ ]:
df.head(10)

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [ ]:
df.shape

(100000, 4)

In [ ]:
df['user_id'].nunique()

943

In [ ]:
df['item_id'].nunique()

1682

In [ ]:
#second dataset
movies_title = pd.read_csv('/content/gdrive/MyDrive/RWDS/ml-100k/ml-100k/u.item',sep = '\|',header = None)
movies_title.shape

(1682, 24)

In [ ]:
movies_title.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


We are only considering movie name and item id. So, we remove the remaining columns


In [ ]:
#first two columns
movies = movies_title[[0,1]]
movies.columns = ['item_id','title']
movies.tail()

,item_id,title
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)
1681,1682,Scream of Stone (Schrei aus Stein) (1991)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Merging both datasets using item_id as common

In [ ]:
df = pd.merge(df,movies,on = 'item_id')
df.tail()

,user_id,item_id,rating,timestamp,title
99995,840,1674,4,891211682,Mamma Roma (1962)
99996,655,1640,3,888474646,"Eighth Day, The (1996)"
99997,655,1637,3,888984255,Girls Town (1996)
99998,655,1630,3,887428735,"Silence of the Palace, The (Saimt el Qusur) (1..."
99999,655,1641,3,887427810,Dadetown (1995)


In [ ]:
#Finding average rating for a movie
ratings = pd.DataFrame(df.groupby('title').mean()['rating'])
ratings.head()

,rating
title,
'Til There Was You (1997),2.333333
1-900 (1994),2.600000
101 Dalmatians (1996),2.908257
12 Angry Men (1957),4.344000
187 (1997),3.024390


In [ ]:
#Number of ratings
ratings['num_of_ratings'] = pd.DataFrame(df.groupby('title').count()['rating'])
ratings.head()

,rating,num_of_ratings
title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41


Creating the recommendar system

In [ ]:
movie_mat = df.pivot_table(index = "user_id",columns = "title",values = "rating")
movie_mat.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",8 1/2 (1963),8 Heads in a Duffel Bag (1997),8 Seconds (1994),A Chef in Love (1996),Above the Rim (1994),Absolute Power (1997),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Across the Sea of Time (1995),Addams Family Values (1993),Addicted to Love (1997),"Addiction, The (1995)","Adventures of Pinocchio, The (1996)","Adventures of Priscilla, Queen of the Desert, The (1994)","Adventures of Robin Hood, The (1938)","Affair to Remember, An (1957)","African Queen, The (1951)",Afterglow (1997),"Age of Innocence, The (1993)",Aiqing wansui (1994),Air Bud (1997),Air Force One (1997),"Air Up There, The (1994)",Airheads (1994),Akira (1988),Aladdin (1992),Aladdin and the King of Thieves (1996),Alaska (1996),Albino Alligator (1996),...,"Whole Wide World, The (1996)",Widows' Peak (1994),"Wife, The (1995)",Wild America (1997),Wild Bill (1995),"Wild Bunch, The (1969)",Wild Reeds (1994),Wild Things (1998),William Shakespeare's Romeo and Juliet (1996),Willy Wonka and the Chocolate Factory (1971),Window to Paris (1994),Wings of Courage (1995),Wings of Desire (1987),"Wings of the Dove, The (1997)",Winnie the Pooh and the Blustery Day (1968),"Winter Guest, The (1997)",Wishmaster (1997),With Honors (1994),Withnail and I (1987),Witness (1985),"Wizard of Oz, The (1939)",Wolf (1994),"Woman in Question, The (1950)","Women, The (1939)","Wonderful, Horrible Life of Leni Riefenstahl, The (1993)",Wonderland (1997),"Wooden Man's Bride, The (Wu Kui) (1994)","World of Apu, The (Apur Sansar) (1959)","Wrong Trousers, The (1993)",Wyatt Earp (1994),Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,� k�ldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN


In [ ]:
#We will remove NaN values and let we take an example of movie 39 steps,The(1935)
steps_user_ratings = movie_mat['39 Steps, The (1935)']
steps_user_ratings.head(10)

user_id
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     4.0
8     NaN
9     4.0
10    4.0
Name: 39 Steps, The (1935), dtype: float64

In [ ]:
#Correletion of movie with movie matrix:
similar_to_steps = movie_mat.corrwith(steps_user_ratings)
similar_to_steps

title
'Til There Was You (1997)                     NaN
1-900 (1994)                                  NaN
101 Dalmatians (1996)                    0.111111
12 Angry Men (1957)                      0.457176
187 (1997)                               1.000000
                                           ...   
Young Guns II (1990)                    -0.054554
Young Poisoner's Handbook, The (1995)    0.707107
Zeus and Roxanne (1997)                       NaN
unknown                                       NaN
� k�ldum klaka (Cold Fever) (1994)            NaN
Length: 1664, dtype: float64

In [ ]:
#Remove NaN values:
corr_steps = pd.DataFrame(similar_to_steps,columns = ["correlation"])
corr_steps.dropna(inplace = True)

In [ ]:
corr_steps.sort_values('correlation',ascending=False).head(10)

,correlation
title,
Dangerous Minds (1995),1.0
Welcome To Sarajevo (1997),1.0
Roommates (1995),1.0
Bhaji on the Beach (1993),1.0
Naked (1993),1.0
"Mark of Zorro, The (1940)",1.0
Striptease (1996),1.0
Fire Down Below (1997),1.0
Batman & Robin (1997),1.0


Here the comparison is not proper, to make the correlation accurate we need to setup a threshold of 100 for no.of ratings

In [ ]:
#Setting a threshold
corr_steps = corr_steps.join(ratings['num_of_ratings'])
corr_steps

,correlation,num_of_ratings
title,,
101 Dalmatians (1996),0.111111,109
12 Angry Men (1957),0.457176,125
187 (1997),1.000000,41
2 Days in the Valley (1996),0.000000,93
"20,000 Leagues Under the Sea (1954)",-0.038127,72
...,...,...
Year of the Horse (1997),1.000000,7
Young Frankenstein (1974),0.179196,200
Young Guns (1988),-0.337691,101


In [ ]:
corr_steps[corr_steps['num_of_ratings']>100].sort_values('correlation',ascending = False)

,correlation,num_of_ratings
title,,
Crash (1996),0.888523,128
Con Air (1997),0.801784,137
"Frighteners, The (1996)",0.759257,115
Swingers (1996),0.726300,157
"River Wild, The (1994)",0.672131,146
...,...,...
Contact (1997),-0.538617,509
Batman Forever (1995),-0.681385,114
"Peacemaker, The (1997)",-0.733359,136


In [ ]:
#Now we make a generalised function for recommending
def predict_movies(movie_name):
  movie_user_ratings = movie_mat[movie_name]
  similar_to_movie = movie_mat.corrwith(movie_user_ratings)
  corr_movie = pd.DataFrame(similar_to_movie,columns = ['correlation'])
  corr_movie.dropna(inplace=True)
  corr_movie = corr_movie.join(ratings['num_of_ratings'])
  predictions = corr_movie[corr_movie['num_of_ratings']>100].sort_values('correlation',ascending = False)
  return predictions

In [ ]:
predict_my_movie = predict_movies('Con Air (1997)')
predict_my_movie.head(15)

,correlation,num_of_ratings
title,,
Con Air (1997),1.000000,137
"Rainmaker, The (1997)",0.629809,145
My Left Foot (1989),0.616057,121
"Philadelphia Story, The (1940)",0.601457,104
Ghost (1990),0.568964,170
"Great Escape, The (1963)",0.566947,124
"Long Kiss Goodnight, The (1996)",0.565832,185
Young Guns (1988),0.557001,101
Seven Years in Tibet (1997),0.549505,155
